In [1]:
import pandas as pd
from tensorflow import keras
model = keras.models.load_model('Lstm_model')
#Loading the pre-trained LSTM Model

In [2]:
idiom=pd.read_excel('idiom.xlsx')
#Loading the dataset, which is just the idioms column

In [3]:
idiom
#Checking/displaying the dataset

,idioms
0,off the beaten track
1,in the running
2,give someone the creeps
3,do someone proud
4,take root
...,...
1733,there's no fool like an old fool
1734,top banana
1735,all dressed up and nowhere to go
1736,keep your nose clean


In [4]:
text=input("the trees have taken root")
#User input to test the classifier!

the trees have taken rootthe trees take root


In [5]:
idiom_temp=''
for i in range(idiom.shape[0]):
    if idiom.idioms[i] in text:
        idiom_temp=idiom.idioms[i]
#Detecting in the user input if there is a present idiom, !

In [6]:
idiom_temp
#Printing out if there is a present idiom!

'take root'

In [7]:
if idiom_temp=='':
    print('Potenial non idiom')
else:
    print('Potential idiom')
#Console output of classification of the potential idiom.

Potential idiom


In [8]:
import torch
import timeit
from transformers import BertTokenizer, BertModel
model1 = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True)
from functools import reduce
import numpy as np


# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt

#Importing all the necessary libraries to run the code!

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
#Initialisting and running the BERT Model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
recor=[]
# text='off the beaten track'

marked_text = "[CLS] " + text + " [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text) #
if len(tokenized_text)>512:
    tokenized_text=tokenized_text[:511]
    tokenized_text.append(" [SEP]")


# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
#         for tup in zip(tokenized_text, indexed_tokens):
#             print('{:<12} {:>6,}'.format(tup[0], tup[1]))

# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

#         print (segments_ids)

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)



# Put the model in "evaluation" mode, meaning feed-forward operation.
model1.eval()

# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model1(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

#         print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

#         print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

#         print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

#         print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

# For the 5th token in our sentence, select its feature values from layer 5.
token_i = 5
layer_i = 5
vec = hidden_states[layer_i][batch_i][token_i]

# Plot the values as a histogram to show their distribution.
# plt.figure(figsize=(10,10))
#  plt.hist(vec, bins=200)
# plt.show()

# `hidden_states` is a Python list.
#         print('      Type of hidden_states: ', type(hidden_states))

# Each layer in the list is a torch tensor.
#         print('Tensor shape for each layer: ', hidden_states[0].size())

# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)
token_embeddings.size()

# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

#         print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)

    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

#         print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

# `hidden_states` has shape [13 x 1 x 22 x 768]

# `token_vecs` is a tensor with shape [22 x 768]
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)

#         print ("Our final sentence embedding vector of shape:", sentence_embedding.size())

#         for i, token_str in enumerate(tokenized_text):
#           print (i, token_str)
    


try:
    o=len(idiom_temp.split())
    p=0
    for i in range(len(tokenized_text)-o+1):
        if ' '.join(tokenized_text[i:i+o])==idiom_temp:
            p=i
            break
        else:
            p='error'
    str(tokenized_text[i])
    tokenized_text[p],tokenized_text[p+o-1]
    k=[]
    for i in range(p,p+o,+1):
        k.append(token_vecs_sum[i][:5].tolist()) 
        #Taking the first 5 layers out of the 769 as they contain the most amount of information, relevant to classification in my opinion.
    k=reduce(lambda z, y :z + y, k)
#         mean_tensor=(float(sum(list(token_vecs_sum[i][:5])))+float(sum(list(token_vecs_sum[i+1][:5])))+float(sum(list(token_vecs_sum[i+2][:5]))))/3
    recor.append(k)
except:
    recor.append('error')

In [10]:
for i in range(45-len(k)):
    k.append(0)
#The idiom can be a maximum of 9 words and the if we have 5 layers per word, the max layers we could take would be 9 * 5 = 45.
#So if we have an idiom of length 20 words for example, the layers after the 45-th 
#would be nothing so we just add zeros so that the LSTM model doesn't cause an error.

In [11]:
x_test=np.array([k])
x_test=np.reshape(x_test,(x_test.shape[0],1,x_test.shape[1]))
#Reshaping as per the LSTM Model input standard requirements

In [12]:
y_pred=model.predict(x_test)
#The prediction function we invoke from the LSTM Model.

In [13]:
y_pred#variable that shows how idiomatic or literal the sentece is. By rounding we get a definitive answer of whether it is an idiom or literal!
#Printing out the prediction, which can be a number from 0 to 1.

array([[0.01160494]], dtype=float32)

In [14]:
if round(y_pred[0][0])==0:
    print('The input is likely to be contextually idiomatic.')
elif round(y_pred[0][0])==1:
    print('The input is likely contextually literal.')
#Final classification/prediction, where if the number in y_pred can be rounded to either 2 extremes, it is 
#and we coclude from that whether the input is idiomatic or literal.

The input is likely to be contextually idiomatic.
